In [62]:
import numpy as np
import os
import sys

import wave
import copy
import math

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input, Flatten,Merge, Embedding, Convolution1D, Dropout, Bidirectional, Concatenate
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.normalization import BatchNormalization
from sklearn.preprocessing import label_binarize
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence


from features import *
from helper import *

In [2]:
#################
#  Load data    #
#################
code_path = os.path.dirname(os.path.realpath(os.getcwd()))
emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
data_path = code_path + "/../../"
sessions = ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']
framerate = 16000
import pickle
with open('./data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [49]:
#################
#  Text feature #
#################
text = []

for ses_mod in data2:
    text.append(' '.join(ses_mod['transcription']['ind_ts']))

MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
x_train_text = []

x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

In [51]:
# get glove pre trained embedding 
import codecs
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

file_loc = '/nfs/project/wangyun' + '/glove.42B.300d.txt'

print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 2735 unique tokens
/nfs/project/wangyun/glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 90


In [3]:
##################
# Speech feature #
##################
def calculate_features(frames, freq, options):
    window_sec = 0.02
    window_n = int(freq * window_sec)

    st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)
    print("st_f", np.shape(st_f))
    if st_f.shape[1] > 2:
        i0 = 1
        i1 = st_f.shape[1] - 1
        if i1 - i0 < 1:
            i1 = i0 + 1
        
        deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
        for i in range(i0, i1):
            i_left = i - 1
            i_right = i + 1
            deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
        return deriv_st_f
    elif st_f.shape[1] == 2:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f
    else:
        deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
        deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
        return deriv_st_f

In [4]:
x_train_speech = []

counter = 0
for ses_mod in data2:
    x_head = ses_mod['signal']
    print("id", ses_mod["id"])
    print("x_head", np.shape(x_head))
    st_features = calculate_features(x_head, framerate, None)
    x_train_speech.append( st_features )
    counter+=1
    if(counter%100==0):
        print(counter)

id Ses01F_impro01_F000
x_head (31130,)
st_f (34, 193)
id Ses01F_impro01_F001
x_head (22120,)
st_f (34, 137)
id Ses01F_impro01_F002
x_head (50085,)
st_f (34, 312)
id Ses01F_impro01_F005
x_head (64480,)
st_f (34, 402)
id Ses01F_impro01_F012
x_head (44000,)
st_f (34, 274)
id Ses01F_impro01_F014
x_head (81120,)
st_f (34, 506)
id Ses01F_impro01_M011
x_head (115840,)
st_f (34, 723)
id Ses01F_impro01_M013
x_head (96480,)
st_f (34, 602)
id Ses01F_impro02_F000
x_head (149300,)
st_f (34, 932)
id Ses01F_impro02_F001
x_head (46880,)
st_f (34, 292)
id Ses01F_impro02_F002
x_head (91577,)
st_f (34, 571)
id Ses01F_impro02_F003
x_head (90441,)
st_f (34, 564)
id Ses01F_impro02_F004
x_head (92320,)
st_f (34, 576)
id Ses01F_impro02_F005
x_head (40960,)
st_f (34, 255)
id Ses01F_impro02_F006
x_head (67441,)
st_f (34, 420)
id Ses01F_impro02_F008
x_head (62191,)
st_f (34, 387)
id Ses01F_impro02_F010
x_head (68000,)
st_f (34, 424)
id Ses01F_impro02_F012
x_head (48488,)
st_f (34, 302)
id Ses01F_impro02_F013
x_h

st_f (34, 667)
id Ses01F_impro06_F006
x_head (192640,)
st_f (34, 1203)
id Ses01F_impro06_F007
x_head (128000,)
st_f (34, 799)
id Ses01F_impro06_F008
x_head (158560,)
st_f (34, 990)
id Ses01F_impro06_F009
x_head (121280,)
st_f (34, 757)
id Ses01F_impro06_F010
x_head (93440,)
st_f (34, 583)
id Ses01F_impro06_F011
x_head (241280,)
st_f (34, 1507)
id Ses01F_impro06_F012
x_head (143520,)
st_f (34, 896)
id Ses01F_impro06_F013
x_head (24520,)
st_f (34, 152)
id Ses01F_impro06_F014
x_head (31840,)
st_f (34, 198)
id Ses01F_impro06_F015
x_head (47704,)
st_f (34, 297)
id Ses01F_impro06_F016
x_head (121560,)
st_f (34, 758)
id Ses01F_impro06_F018
x_head (115200,)
st_f (34, 719)
id Ses01F_impro06_F019
x_head (26560,)
st_f (34, 165)
id Ses01F_impro06_F020
x_head (242720,)
st_f (34, 1516)
id Ses01F_impro06_F021
x_head (265360,)
st_f (34, 1657)
id Ses01F_impro06_F022
x_head (78560,)
st_f (34, 490)
id Ses01F_impro06_F023
x_head (64480,)
st_f (34, 402)
id Ses01F_impro06_F024
x_head (110400,)
st_f (34, 689

st_f (34, 296)
id Ses01F_script02_1_F007
x_head (29120,)
st_f (34, 181)
300
id Ses01F_script02_1_F008
x_head (21920,)
st_f (34, 136)
id Ses01F_script02_1_F010
x_head (30720,)
st_f (34, 191)
id Ses01F_script02_1_F015
x_head (37193,)
st_f (34, 231)
id Ses01F_script02_1_F024
x_head (41760,)
st_f (34, 260)
id Ses01F_script02_1_F025
x_head (39520,)
st_f (34, 246)
id Ses01F_script02_1_M002
x_head (48640,)
st_f (34, 303)
id Ses01F_script02_1_M007
x_head (56480,)
st_f (34, 352)
id Ses01F_script02_1_M008
x_head (24800,)
st_f (34, 154)
id Ses01F_script02_1_M009
x_head (27520,)
st_f (34, 171)
id Ses01F_script02_1_M010
x_head (18080,)
st_f (34, 112)
id Ses01F_script02_1_M011
x_head (107545,)
st_f (34, 671)
id Ses01F_script02_1_M013
x_head (63797,)
st_f (34, 397)
id Ses01F_script02_1_M014
x_head (167806,)
st_f (34, 1047)
id Ses01F_script02_1_M015
x_head (107224,)
st_f (34, 669)
id Ses01F_script02_1_M016
x_head (92160,)
st_f (34, 575)
id Ses01F_script02_1_M020
x_head (30240,)
st_f (34, 188)
id Ses01

st_f (34, 221)
id Ses01M_impro01_M020
x_head (57920,)
st_f (34, 361)
id Ses01M_impro01_M025
x_head (53520,)
st_f (34, 333)
id Ses01M_impro01_M030
x_head (17920,)
st_f (34, 111)
id Ses01M_impro01_M031
x_head (42720,)
st_f (34, 266)
id Ses01M_impro01_M032
x_head (88160,)
st_f (34, 550)
id Ses01M_impro02_F000
x_head (54341,)
st_f (34, 338)
id Ses01M_impro02_F009
x_head (52800,)
st_f (34, 329)
id Ses01M_impro02_F010
x_head (52160,)
st_f (34, 325)
id Ses01M_impro02_F013
x_head (64960,)
st_f (34, 405)
id Ses01M_impro02_F015
x_head (43040,)
st_f (34, 268)
id Ses01M_impro02_F017
x_head (79040,)
st_f (34, 493)
id Ses01M_impro02_F019
x_head (36680,)
st_f (34, 228)
id Ses01M_impro02_M000
x_head (52829,)
st_f (34, 329)
id Ses01M_impro02_M001
x_head (60160,)
st_f (34, 375)
id Ses01M_impro02_M002
x_head (68800,)
st_f (34, 429)
id Ses01M_impro02_M003
x_head (43440,)
st_f (34, 270)
id Ses01M_impro02_M007
x_head (136320,)
st_f (34, 851)
id Ses01M_impro02_M009
x_head (46760,)
st_f (34, 291)
id Ses01M_im

st_f (34, 698)
id Ses01M_impro05_M026
x_head (107520,)
st_f (34, 671)
id Ses01M_impro05_M027
x_head (98920,)
st_f (34, 617)
id Ses01M_impro05_M028
x_head (85200,)
st_f (34, 531)
600
id Ses01M_impro05_M031
x_head (38840,)
st_f (34, 241)
id Ses01M_impro06_F000
x_head (41352,)
st_f (34, 257)
id Ses01M_impro06_F001
x_head (50040,)
st_f (34, 311)
id Ses01M_impro06_F002
x_head (37120,)
st_f (34, 231)
id Ses01M_impro06_F003
x_head (44640,)
st_f (34, 278)
id Ses01M_impro06_F004
x_head (45488,)
st_f (34, 283)
id Ses01M_impro06_F005
x_head (33046,)
st_f (34, 205)
id Ses01M_impro06_F006
x_head (40455,)
st_f (34, 251)
id Ses01M_impro06_F007
x_head (35421,)
st_f (34, 220)
id Ses01M_impro06_F008
x_head (30400,)
st_f (34, 189)
id Ses01M_impro06_F009
x_head (63520,)
st_f (34, 396)
id Ses01M_impro06_F010
x_head (69880,)
st_f (34, 435)
id Ses01M_impro06_F011
x_head (24640,)
st_f (34, 153)
id Ses01M_impro06_F012
x_head (21800,)
st_f (34, 135)
id Ses01M_impro06_F013
x_head (47200,)
st_f (34, 294)
id Ses01

...

st_f (34, 2903)
id Ses05F_script02_1_M034
x_head (69120,)
st_f (34, 431)
id Ses05F_script02_2_F005
x_head (90357,)
st_f (34, 563)
id Ses05F_script02_2_F013
x_head (187908,)
st_f (34, 1173)
id Ses05F_script02_2_F014
x_head (386020,)
st_f (34, 2411)
id Ses05F_script02_2_F016
x_head (95704,)
st_f (34, 597)
id Ses05F_script02_2_F017
x_head (95456,)
st_f (34, 595)
4300
id Ses05F_script02_2_F018
x_head (29407,)
st_f (34, 182)
id Ses05F_script02_2_F019
x_head (28336,)
st_f (34, 176)
id Ses05F_script02_2_F025
x_head (80398,)
st_f (34, 501)
id Ses05F_script02_2_F026
x_head (84092,)
st_f (34, 524)
id Ses05F_script02_2_F027
x_head (70038,)
st_f (34, 436)
id Ses05F_script02_2_F028
x_head (309029,)
st_f (34, 1930)
id Ses05F_script02_2_F029
x_head (32080,)
st_f (34, 199)
id Ses05F_script02_2_F030
x_head (120832,)
st_f (34, 754)
id Ses05F_script02_2_F032
x_head (264119,)
st_f (34, 1649)
id Ses05F_script02_2_M000
x_head (60960,)
st_f (34, 380)
id Ses05F_script02_2_M002
x_head (96192,)
st_f (34, 600)
i

st_f (34, 490)
id Ses05M_impro02_M024
x_head (51040,)
st_f (34, 318)
id Ses05M_impro02_M025
x_head (150080,)
st_f (34, 937)
id Ses05M_impro02_M027
x_head (30720,)
st_f (34, 191)
id Ses05M_impro02_M028
x_head (172960,)
st_f (34, 1080)
id Ses05M_impro02_M029
x_head (36640,)
st_f (34, 228)
id Ses05M_impro02_M030
x_head (79520,)
st_f (34, 496)
id Ses05M_impro03_F000
x_head (23040,)
st_f (34, 143)
id Ses05M_impro03_F002
x_head (16320,)
st_f (34, 101)
id Ses05M_impro03_F003
x_head (55200,)
st_f (34, 344)
id Ses05M_impro03_F004
x_head (44000,)
st_f (34, 274)
id Ses05M_impro03_F006
x_head (22080,)
st_f (34, 137)
id Ses05M_impro03_F007
x_head (67840,)
st_f (34, 423)
id Ses05M_impro03_F014
x_head (41280,)
st_f (34, 257)
id Ses05M_impro03_F015
x_head (28960,)
st_f (34, 180)
id Ses05M_impro03_F018
x_head (18400,)
st_f (34, 114)
id Ses05M_impro03_F019
x_head (47520,)
st_f (34, 296)
id Ses05M_impro03_F023
x_head (31360,)
st_f (34, 195)
id Ses05M_impro03_F026
x_head (32960,)
st_f (34, 205)
id Ses05M_

st_f (34, 312)
id Ses05M_impro07_F030
x_head (14239,)
st_f (34, 87)
id Ses05M_impro07_F032
x_head (17280,)
st_f (34, 107)
id Ses05M_impro07_F033
x_head (21280,)
st_f (34, 132)
id Ses05M_impro07_F037
x_head (19200,)
st_f (34, 119)
id Ses05M_impro07_F038
x_head (22560,)
st_f (34, 140)
id Ses05M_impro07_F040
x_head (24480,)
st_f (34, 152)
id Ses05M_impro07_F041
x_head (35360,)
st_f (34, 220)
id Ses05M_impro07_F042
x_head (79680,)
st_f (34, 497)
id Ses05M_impro07_F044
x_head (61120,)
st_f (34, 381)
4600
id Ses05M_impro07_M000
x_head (37600,)
st_f (34, 234)
id Ses05M_impro07_M001
x_head (41120,)
st_f (34, 256)
id Ses05M_impro07_M002
x_head (48721,)
st_f (34, 303)
id Ses05M_impro07_M003
x_head (37679,)
st_f (34, 234)
id Ses05M_impro07_M004
x_head (53760,)
st_f (34, 335)
id Ses05M_impro07_M005
x_head (21760,)
st_f (34, 135)
id Ses05M_impro07_M006
x_head (18080,)
st_f (34, 112)
id Ses05M_impro07_M007
x_head (25440,)
st_f (34, 158)
id Ses05M_impro07_M011
x_head (105120,)
st_f (34, 656)
id Ses05

st_f (34, 297)
id Ses05M_script01_2_F017
x_head (23840,)
st_f (34, 148)
id Ses05M_script01_2_M000
x_head (39841,)
st_f (34, 248)
id Ses05M_script01_2_M001
x_head (88640,)
st_f (34, 553)
id Ses05M_script01_2_M003
x_head (53120,)
st_f (34, 331)
id Ses05M_script01_2_M015
x_head (54080,)
st_f (34, 337)
id Ses05M_script01_2_M016
x_head (63680,)
st_f (34, 397)
id Ses05M_script01_3_F000
x_head (47200,)
st_f (34, 294)
id Ses05M_script01_3_F003
x_head (59680,)
st_f (34, 372)
id Ses05M_script01_3_F006
x_head (56800,)
st_f (34, 354)
id Ses05M_script01_3_F007
x_head (82399,)
st_f (34, 513)
id Ses05M_script01_3_F010
x_head (42720,)
st_f (34, 266)
id Ses05M_script01_3_F013
x_head (118400,)
st_f (34, 739)
id Ses05M_script01_3_F015
x_head (41600,)
st_f (34, 259)
id Ses05M_script01_3_F016
x_head (120486,)
st_f (34, 752)
id Ses05M_script01_3_F019
x_head (66880,)
st_f (34, 417)
id Ses05M_script01_3_F020
x_head (35360,)
st_f (34, 220)
id Ses05M_script01_3_F021
x_head (33569,)
st_f (34, 208)
id Ses05M_scri

st_f (34, 238)
id Ses05M_script03_2_F002
x_head (58240,)
st_f (34, 363)
id Ses05M_script03_2_F004
x_head (54881,)
st_f (34, 342)
id Ses05M_script03_2_F005
x_head (23360,)
st_f (34, 145)
id Ses05M_script03_2_F006
x_head (52160,)
st_f (34, 325)
id Ses05M_script03_2_F009
x_head (26240,)
st_f (34, 163)
id Ses05M_script03_2_F012
x_head (41440,)
st_f (34, 258)
id Ses05M_script03_2_F015
x_head (44800,)
st_f (34, 279)
id Ses05M_script03_2_F017
x_head (22720,)
st_f (34, 141)
id Ses05M_script03_2_F018
x_head (45600,)
st_f (34, 284)
id Ses05M_script03_2_F023
x_head (29760,)
st_f (34, 185)
id Ses05M_script03_2_F026
x_head (29440,)
st_f (34, 183)
id Ses05M_script03_2_F028
x_head (46080,)
st_f (34, 287)
id Ses05M_script03_2_F034
x_head (68480,)
st_f (34, 427)
id Ses05M_script03_2_F035
x_head (48160,)
st_f (34, 300)
id Ses05M_script03_2_F036
x_head (53440,)
st_f (34, 333)
id Ses05M_script03_2_F037
x_head (85120,)
st_f (34, 531)
id Ses05M_script03_2_F038
x_head (105440,)
st_f (34, 658)
4900
id Ses05M_

In [7]:
with open('./speech_features.pickle', 'wb') as handle:
    pickle.dump(x_train_speech, handle)

In [24]:
# alignment
align = []
for ses_mod in data2:
    align_t = ses_mod['transcription']['align_t']
    align_t_ = [(int(x['SFrm']), int(x['Efrm'])) for x in align_t]
    align.append(align_t_)
    

In [43]:
def avg_frame(frame):
    frame_to, ali = frame
    features = []
    for sf, ef in ali:
        if sf == ef:
            print(sf)
        features.append(np.average(frame_to.T[sf:ef+1].T, axis=1))
    features = np.array(features).T
    print("before pad", np.shape(features))
    features, _ = pad_sequence_into_array(features, maxlen=MAX_SEQUENCE_LENGTH)
    print("after pad", np.shape(features))
    print()
    return features.T
    

In [47]:
x_train_speech_ali = list(map(avg_frame, zip(x_train_speech, align)))

before pad (34, 2)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 24)
after pad (34, 500)

before pad (34, 22)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 18)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 6)
after pad (34,

after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 32)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 13)
after pad (34, 500)

before pad (34, 15)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 26)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 12)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 30)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 25)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 25)
after pad (34, 500)

before pa

after pad (34, 500)

before pad (34, 15)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 37)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 26)
after pad (34, 500)

before pad (34, 35)
after pad (34, 500)

before pad (34, 15)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 24)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 43)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 13)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before p

after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 13)
after pad (34, 500)

before pad (34, 20)
after pad (34, 500)

before pad (34, 14)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 24)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 17)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 16)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 60)
after pad (34, 500)

before pad (

before pad (34, 23)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 29)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 20)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 13)
after pad (34, 500)

before pad (34, 16)
after pad (34, 500)

before pad (34, 24)
after pad (34, 500)

before pad (34, 16)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 22)
after pad (34, 500)

before pad (34, 20)
after pad (34, 500)

before pad (34, 26)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 16)
after pad (34, 500)

before pad (34, 17)
after pad (34, 500)

before pad (34, 50)
after pad (34, 500)

before pad (34, 12)
after pad (34, 500)

before pad (34, 18)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 37)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 44)


...

after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 36)
after pad (34, 500)

before pad (34, 29)
after pad (34, 500)

before pad (34, 28)
after pad (34, 500)

before pad (34, 28)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 20)
after pad (34, 500)

before pad (34, 16)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 48)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 12)
after pad (34, 500)

before pad (34, 33)
after pad (34, 500)

before pad (34, 20)
after pad (34, 500)

before pad (34, 35)
after pad (34, 500)

before pad (34, 39)
after pad (34, 500)

before pad (34, 32)
after pad (34, 500)

before pad (34, 22)
after pad (34, 500)

before pad (34, 56)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

befo

after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 16)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 28)
after pad (34, 500)

before pad (34, 15)
after pad (34, 500)

before pad (34, 21)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 13)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 21)
after pad (34, 500)

before pad (34, 21)
after pad (34, 500)

before pad (34, 48)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad 

after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 9)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 13)
after pad (34, 500)

before pad (34, 15)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34,

after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 2)
after pad (34, 500)

before pad (34, 14)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 18)
after pad (34, 500)

before pad (34, 14)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 12)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 15)
after pad (34, 500)

before pad (34, 23)
after pad (34, 500)

before pad (34, 1)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 13)
after pad (34, 500)

before pad (34, 11)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad 

after pad (34, 500)

before pad (34, 3)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 23)
after pad (34, 500)

before pad (34, 20)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 24)
after pad (34, 500)

before pad (34, 24)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 30)
after pad (34, 500)

before pad (34, 10)
after pad (34, 500)

before pad (34, 17)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 33)
after pad (34, 500)

before pad (34, 6)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 8)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad (34, 7)
after pad (34, 500)

before pad (34, 4)
after pad (34, 500)

before pad (34, 5)
after pad (34, 500)

before pad

In [48]:
x_train_speech_ali = np.array(x_train_speech_ali)
np.shape(x_train_speech_ali)

(4935, 500, 34)

In [52]:
##########
#    Y
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = label_binarize(Y,emotions_used)

Y.shape

(4935, 4)

In [64]:
##############
##  Model
def model_1(optimizer='adam'):
    input_speech = Input(shape=(MAX_SEQUENCE_LENGTH, 34))
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
    #model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
    layer = Embedding(nb_words,
                  EMBEDDING_DIM,
                  weights = [g_word_embedding_matrix],
                  input_length = MAX_SEQUENCE_LENGTH,
                  trainable = True)(input_text)
    layer = Concatenate(axis=-1)([layer, input_speech])
    layer = Convolution1D(256, 3, border_mode='same')(layer)
    layer = Dropout(0.2)(layer)
    layer = Activation('relu')(layer)
    layer = Convolution1D(128, 3, border_mode='same')(layer)
    layer = Dropout(0.2)(layer)
    layer = Activation('relu')(layer)
    layer = Convolution1D(64, 3, border_mode='same')(layer)
    layer = Dropout(0.2)(layer)
    layer = Activation('relu')(layer)
    layer = Convolution1D(32, 3, border_mode='same')(layer)
    layer = Dropout(0.2)(layer)
    layer = Activation('relu')(layer)
    layer = Flatten()(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(256)(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.2)(layer)
    layer = Dense(4)(layer)
    prediction = Activation('softmax')(layer)

    model = Model(inputs=[input_speech, input_text], outputs=prediction)
    model.compile(loss='categorical_crossentropy',optimizer=optimizer ,metrics=['acc'])
    return model

In [65]:
model = model_1()
model.summary()

/home/luban/miniconda3/envs/nluml/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  del sys.path[0]
/home/luban/miniconda3/envs/nluml/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  app.launch_new_instance()
/home/luban/miniconda3/envs/nluml/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
/home/luban/miniconda3/envs/nluml/lib/python3.6/site-packages/ipykernel_launcher.py:22: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_6 (InputLayer)             (None, 500)           0                                            
____________________________________________________________________________________________________
embedding_6 (Embedding)          (None, 500, 300)      820800      input_6[0][0]                    
____________________________________________________________________________________________________
input_5 (InputLayer)             (None, 500, 34)       0                                            
____________________________________________________________________________________________________
concatenate_2 (Concatenate)      (None, 500, 334)      0           embedding_6[0][0]                
                                                                   input_5[0][0]           

In [66]:
hist = model.fit([x_train_speech_ali, x_train_text], Y, 
                 batch_size=100, epochs=125, verbose=1, 
                 validation_split=0.2)

Train on 3948 samples, validate on 987 samples
Epoch 1/125
3948/3948 [==============================] - 5s - loss: 1.3739 - acc: 0.3313 - val_loss: 1.3591 - val_acc: 0.3576
Epoch 2/125
3948/3948 [==============================] - 2s - loss: 1.3430 - acc: 0.3556 - val_loss: 1.3314 - val_acc: 0.3749
Epoch 3/125
3948/3948 [==============================] - 2s - loss: 1.2170 - acc: 0.4445 - val_loss: 1.2901 - val_acc: 0.4823
Epoch 4/125
3948/3948 [==============================] - 2s - loss: 1.0173 - acc: 0.5709 - val_loss: 1.0907 - val_acc: 0.5765
Epoch 5/125
3948/3948 [==============================] - 2s - loss: 0.7970 - acc: 0.6727 - val_loss: 1.0382 - val_acc: 0.6008
Epoch 6/125
3948/3948 [==============================] - 2s - loss: 0.6192 - acc: 0.7553 - val_loss: 1.1259 - val_acc: 0.6272
Epoch 7/125
3948/3948 [==============================] - 2s - loss: 0.5063 - acc: 0.8072 - val_loss: 1.0798 - val_acc: 0.6535
Epoch 8/125
3948/3948 [==============================] - 2s - loss: 0.4

In [67]:
def model_2():
    input_speech = Input(shape=(MAX_SEQUENCE_LENGTH, 34))
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
    layer = Embedding(nb_words,
                      EMBEDDING_DIM,
                      weights = [g_word_embedding_matrix],
                      input_length = MAX_SEQUENCE_LENGTH,
                      trainable = True)(input_text)
    layer = Concatenate(axis=-1)([layer, input_speech])
    layer = LSTM(512, return_sequences=True)(layer)
    layer = LSTM(256, return_sequences=False)(layer)
    layer = Dense(512)(layer)
    layer = Activation('relu')(layer)
    layer = Dense(4)(layer)
    prediction = Activation('softmax')(layer)
    
    model = Model(inputs=[input_speech, input_text], outputs=prediction)
    model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])
    
    return model

In [68]:
model_2 = model_2()
model_2.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 500)           0                                            
____________________________________________________________________________________________________
embedding_7 (Embedding)          (None, 500, 300)      820800      input_8[0][0]                    
____________________________________________________________________________________________________
input_7 (InputLayer)             (None, 500, 34)       0                                            
____________________________________________________________________________________________________
concatenate_3 (Concatenate)      (None, 500, 334)      0           embedding_7[0][0]                
                                                                   input_7[0][0]           

In [74]:
hist2 = model_2.fit([x_train_speech_ali, x_train_text], Y, 
                 batch_size=100, epochs=30, verbose=1, 
                 validation_split=0.2)

Train on 3948 samples, validate on 987 samples
Epoch 1/30
3948/3948 [==============================] - 61s - loss: 1.3085 - acc: 0.3860 - val_loss: 1.2755 - val_acc: 0.3891
Epoch 2/30
3948/3948 [==============================] - 61s - loss: 1.0594 - acc: 0.5537 - val_loss: 1.0443 - val_acc: 0.5694
Epoch 3/30
3948/3948 [==============================] - 61s - loss: 0.8404 - acc: 0.6555 - val_loss: 0.9654 - val_acc: 0.6190
Epoch 4/30
3948/3948 [==============================] - 62s - loss: 0.6858 - acc: 0.7277 - val_loss: 0.9133 - val_acc: 0.6322
Epoch 5/30
3948/3948 [==============================] - 62s - loss: 0.5601 - acc: 0.7824 - val_loss: 0.9498 - val_acc: 0.6393
Epoch 6/30
3948/3948 [==============================] - 61s - loss: 0.4824 - acc: 0.8146 - val_loss: 1.0677 - val_acc: 0.6444
Epoch 7/30
3948/3948 [==============================] - 61s - loss: 0.4072 - acc: 0.8397 - val_loss: 1.1171 - val_acc: 0.6322
Epoch 8/30
3948/3948 [==============================] - 61s - loss: 0.3

In [72]:
def model_3():
    input_speech = Input(shape=(MAX_SEQUENCE_LENGTH, 34))
    input_text = Input(shape=(MAX_SEQUENCE_LENGTH,))
    layer = Embedding(2736, 128)(input_text)
    layer = Concatenate(axis=-1)([layer, input_speech])
    layer = LSTM(256, return_sequences=True)(layer)
    layer = LSTM(256, return_sequences=False)(layer)
    layer = Dense(512)(layer)
    layer = Activation('relu')(layer)
    layer = Dense(4)(layer)
    prediction = Activation('softmax')(layer)
    
    model = Model(inputs=[input_speech, input_text], outputs=prediction)
    model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])
    
    return model

In [73]:
model_3 = model_3()
model_3.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_12 (InputLayer)            (None, 500)           0                                            
____________________________________________________________________________________________________
embedding_9 (Embedding)          (None, 500, 128)      350208      input_12[0][0]                   
____________________________________________________________________________________________________
input_11 (InputLayer)            (None, 500, 34)       0                                            
____________________________________________________________________________________________________
concatenate_5 (Concatenate)      (None, 500, 162)      0           embedding_9[0][0]                
                                                                   input_11[0][0]          

In [75]:
hist3 = model_3.fit([x_train_speech_ali, x_train_text], Y, 
                 batch_size=100, epochs=30, verbose=1, 
                 validation_split=0.2)

Train on 3948 samples, validate on 987 samples
Epoch 1/30
3948/3948 [==============================] - 62s - loss: 1.3592 - acc: 0.3366 - val_loss: 1.3371 - val_acc: 0.4245
Epoch 2/30
3948/3948 [==============================] - 60s - loss: 1.1456 - acc: 0.5167 - val_loss: 1.0443 - val_acc: 0.5866
Epoch 3/30
3948/3948 [==============================] - 59s - loss: 0.8057 - acc: 0.6831 - val_loss: 0.9503 - val_acc: 0.6474
Epoch 4/30
3948/3948 [==============================] - 59s - loss: 0.6365 - acc: 0.7599 - val_loss: 0.9891 - val_acc: 0.6464
Epoch 5/30
3948/3948 [==============================] - 58s - loss: 0.5431 - acc: 0.7966 - val_loss: 1.0438 - val_acc: 0.6464
Epoch 6/30
3948/3948 [==============================] - 59s - loss: 0.4750 - acc: 0.8126 - val_loss: 1.1270 - val_acc: 0.6211
Epoch 7/30
3948/3948 [==============================] - 59s - loss: 0.4496 - acc: 0.8240 - val_loss: 1.1769 - val_acc: 0.6190
Epoch 8/30
3948/3948 [==============================] - 60s - loss: 0.4